# Nothing But `True`: Part 2!

## Recap

In part one, I defined an optimization puzzle: How can we most concisely express any integer, purely in terms of Python operators and the "True" expression?

The rules are:
1. Only operators, `True`s and parentheses allowed!
2. Any [Python operators](https://docs.python.org/3/reference/lexical_analysis.html#operators) are allowed, unary or binary.
3. We must be able to construct any integer.
4. Parentheses are allowed at will, but more concise expressions, following spacing conventions, are preferred (let's say as a tiebreaker).

**Allowed**

```
(True + True) = 2
~(True + True + True) = -4
((True + True) ** (True + True + True) ** (True + True)) ^ (True + True) = 514
```

**Not Allowed**

```
(True + True) ** 5 = 32

total = 0
for i in range((True + True + True + True)):
    total += i ** i
--> total = 33
```

Let's lay out some terms:

A **strategy** is a method of generating expressions that evaluate to an arbitrary integer **target**. In part 1, we made Strategy 1 which simply adds (or subtracts) `True`s to make any number, and Strategy 2 which used Strategy 1 to shift bits to build any integer from its binary representation.

The **score** for a particular strategy _and_ target is the number of `True`s needed to express the target using the given strategy. We are aiming to build a strategy which has scores **as low as possible**.

> Can you think of a problem with this scoring approach? I realized the issue last night when doing some experimenting... Hint: Recall we allowed the use of unary operators `~` and `-` (bitwise not and minus). We'll see the problem soon, and then we can revise our scoring technique.

## Code from Part 1

Let's bring back our `strategy_1` and `strategy_2` functions from part 1, as well as our helpers.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

def strategy_1(target: int) -> str:
    if target == 0:
        return "True - True"
    elif target < 0:
        expr = "-True"
        for i in range(abs(target) - 1):
            expr += " - True"
        return expr
    else:
        expr = "True"
        for i in range(target - 1):
            expr += " + True"
        return expr

def strategy_2(target: int) -> str:
    # Convert absolute value to binary and cut off binary prefix
    binary_string = bin(abs(target))[2:]  # Prefix is "0b"
    expr = ""

    num_bits = len(binary_string)
    for i, char in enumerate(binary_string):
        if char == '1':
            bit_shift_count = num_bits - i - 1
            expr += f"(True << {strategy_1(bit_shift_count)}) + "

    expr = expr[:-3]  # Chop off trailing " + "

    if target < 0:
        expr = "-(" + expr + ")"  # Invert the sign for negative numbers

    return expr

def test_strategy(func: Callable, test_targets) -> None:
    for targ in test_targets:
        expr = func(targ)
        result = eval(expr)
        assert result == targ, f"Expression '{expr}' does not evaluate to {targ}! Got {result} instead."
        print(f"{expr} = {result}")

def count_trues(func: Callable, target: int) -> int:
    return func(target).count("True")

## The Optimal Strategy

The strategy I came up with to determine the optimal expression for some targets is as follows.

1. We already have a 'base case': we can make the number 1 with the expression `"True"`.
2. Any other expression is going to use some unary or binary operators on our base case. So we can compute all the values we can reach using our operators on our base case, and the score of the resulting target obtained is equal to the sum of the scores of the operands. For example, we can reach the target `-1` by doing the unary operation `-` on `1`: `-True = -1`. Since a score must be at least 1, we know this must be at least tied for the best way to obtain `-1`. We can continue to look for minimal-score combinations of our existing targets and operators to build up more optimal combos.

First, let's collect all the operators we want to use. I'll discard special and boolean operators because, as we discussed in part 1, they will not be useful.

In [2]:
arithmetic_operators = ['+', '-', '*', '**', '/', '//', '%']
bitwise_operators = ['<<', '>>', '&', '|', '^', '~']

We can group them into unary and binary operators. I'll also distinguish commutative and non-commutative binary operators.

Here, I will also discard the '`/`' (non-integer division) operator since it allows us to obtain non-integers from integers. We don't lose any functionality since the integer division '`//`' operator does the same thing when the operands are valid. All other operators return integers when their operands are both integers.

In [3]:
unary_operators = ['-', '~']
binary_non_commutative = ['<<', '>>', '-', '**', '//', '%']  # '/' removed
binary_commutative = ['+', '*', '&', '|', '^']

Let's make a script which tracks optimal expressions in a certain data structure.

I've opted to use a list[dict[int,str]].

The index in the main list corresponds to the **minimum score** of the target.

The key in the dictionary corresponds to the value of the **target**.

The value in the dictionary, for a given target, is the string **expression** which evaluates to the target.

First, let's calculate all of the targets we can reach with only unary operators. These targets will all have a score of 1.

In [4]:
def get_new_targs_unary(exprs: list[dict[int, str]]) -> list[dict[int, str]]:
    new_exprs: list[dict[int, str]] = [{}] # mutable to store the new stuff

    # Iterate over all targs in dict, at all s-values
    for i, min_s_targs in enumerate(exprs):
        s_val = i + 1
        for targ, expr in min_s_targs.items():

            # Determine all outputs of unary operators
            for u_op in unary_operators:

                # Determine what target we get to using this operator on the given targ
                new_expr = f"{u_op}{expr}"
                new_targ = eval(new_expr)
                new_targ_s_val = s_val
                print(f"Reached target {new_targ} with score {new_targ_s_val}")

                # Add the new target to the new_exprs dictionary
                new_exprs[new_targ_s_val - 1].update({new_targ: new_expr})

        return new_exprs

In [5]:
import warnings
# Python doesn't like it when you do ~True, because it thinks you're doing it by mistake! I am not.
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
min_s_expressions_iters = [[{1: "True"}]] # start with the base case, iter 0

iters = 10
for i in range(iters):

    prev_min_s_expressions = min_s_expressions_iters[-1] # the min s expressions found in the last iteration

    new_min_s_expressions = get_new_targs_unary(prev_min_s_expressions)

    print("Newly found expressions: ", new_min_s_expressions)

    min_s_expressions_iters.append(new_min_s_expressions)

Reached target -1 with score 1
Reached target -2 with score 1
Newly found expressions:  [{-1: '-True', -2: '~True'}]
Reached target 1 with score 1
Reached target 0 with score 1
Reached target 2 with score 1
Reached target 1 with score 1
Newly found expressions:  [{1: '~~True', 0: '~-True', 2: '-~True'}]
Reached target -1 with score 1
Reached target -2 with score 1
Reached target 0 with score 1
Reached target -1 with score 1
Reached target -2 with score 1
Reached target -3 with score 1
Newly found expressions:  [{-1: '~~-True', -2: '--~True', 0: '-~-True', -3: '~-~True'}]
Reached target 1 with score 1
Reached target 0 with score 1
Reached target 2 with score 1
Reached target 1 with score 1
Reached target 0 with score 1
Reached target -1 with score 1
Reached target 3 with score 1
Reached target 2 with score 1
Newly found expressions:  [{1: '~--~True', 0: '--~-True', 2: '~~-~True', -1: '~-~-True', 3: '-~-~True'}]
Reached target -1 with score 1
Reached target -2 with score 1
Reached target

With only 10 iterations, we found a bunch of new targets using only unary operators! Let's combine the results from all the iterations, and take the shortest strings if there are ties

In [8]:
def merge_dicts_with_shortest_string(dict1, dict2):
    """
    Merges two dictionaries with int keys and str values.
    In case of key conflicts, the shortest string is kept.
    """
    merged_dict = dict1.copy()  # Start with a copy of the first dictionary

    for key, value in dict2.items():
        # If the key exists in both dictionaries, choose the shortest string
        if key in merged_dict:
            merged_dict[key] = min(merged_dict[key], value, key=len)
        else:
            merged_dict[key] = value

    return merged_dict

all_min_s_expressions = []

for iter in min_s_expressions_iters:
    for i, dict in enumerate(iter):
        if len(all_min_s_expressions) < len(iter):
            # no corresponding dict exists yet
            all_min_s_expressions.append(dict)
        else:
            all_min_s_expressions[i] = merge_dicts_with_shortest_string(dict, all_min_s_expressions[i])

print("All found expressions: ", all_min_s_expressions)

All found expressions:  [{1: 'True', 0: '~-True', 2: '-~True', -1: '-True', 3: '-~-~True', -2: '~True', 4: '-~-~-~True', -3: '~-~True', 5: '-~-~-~-~True', -4: '~-~-~True', 6: '-~-~-~-~-~True', -5: '~-~-~-~True', -6: '~-~-~-~-~True'}]


I imagine you can see the critical problem now. Our system figures out that **you can actually make any integer by just prepending alternating `-` and `~` operators before the `True`!**

I think this is a pretty awesome outcome, but it kind of breaks the game.

## A Better Scoring System

Maybe you were thinking this from the start, but it's certainly clear now that we need to minimize the use of operators as well as `True`s. The new scoring will be simple: we still allow any number of parentheses, but we score based on `True`s + operators.

At this point, it's becoming clearer that a more structured approach is needed for the optimization, so I'll create a better data structure to track the min-s (minimum score) expressions and optimize them.

# Operators

To start our more organized code, let's define some operators as enums in OOP. This clearly defines all avaliable operators.

In [11]:
from enum import Enum

class Operator(Enum):
    pass # subclasses distinguish unary/binary

class UnaryOperator(Operator):
    MINUS = '-'
    BITWISE_NOT = '~'

class BinaryOperator(Operator):
    pass # subclasses distinguish commutative/noncommutative

class BinaryNonCommutativeOperator(BinaryOperator):
    BIT_SHIFT_LEFT = '<<'
    BIT_SHIFT_RIGHT = '>>'
    SUBTRACT = '-'
    EXPONENTIATE = '**'
    INT_DIVIDE = '//'
    MODULO = '%'

class BinaryCommutativeOperator(BinaryOperator):
    ADD = '+'
    MULTIPLY = '*'
    BITWISE_AND = '&'
    BITWISE_OR = '|'
    BITWISE_XOR = '^'

In [31]:
from typing import Union

class Expression:
    """A string expression that evaluates to an integer."""

    def __init__(self, string: str) -> None:
        if type(self) is Expression:
            raise TypeError("Expression cannot be instantiated directly.")
        self._string: str = string

    def evaluate(self) -> int:
        """Evaluate the expression and return an integer."""
        return int(eval(self._string))

    def __str__(self):
        return self._string

class TrueExpression(Expression):
    """Base case expression. Evaluates to 1."""

    def __init__(self) -> None:
        super().__init__('True')

class CompositeExpression(Expression):
    """Composite expression, composed of at least one operator and one expression."""

    def __init__(self,
                 operator: Union[UnaryOperator, BinaryOperator],
                 expr_1: Expression,
                 expr_2: Expression = None) -> None:
        """Instantiate a CompositeExpression either from a unary or binary operator."""
        if expr_2 is None:  # Unary operator case
            if not isinstance(operator, UnaryOperator):
                raise TypeError("Operator is binary, but only one expression was given!")
            self._string = f"{operator.value}({expr_1._string})"
        else:  # Binary operator case
            if not isinstance(operator, BinaryOperator):
                raise TypeError("Operator is unary, but two expressions were given!")
            self._string = f"({expr_1._string}) {operator.value} ({expr_2._string})"

        super().__init__(self._string)

We've now defined a clear structure and definition for operators and expressions.

Expressions are composed of either:
- Just the expression for 'True' (this is a sort of recursive base case)
- A unary operator acting on an expression
- A binary operator acting on two expressions

In [32]:
t = TrueExpression()
print(f"{t} = {t.evaluate()}")

e1 = CompositeExpression(UnaryOperator.MINUS, t)
print(f"{e1} = {e1.evaluate()}")

# FAILS - negative shift count!
# e2 = CompositeExpression(BinaryNonCommutativeOperator.BIT_SHIFT_LEFT, t, e1)
# print(f"{e2} = {e2.evaluate()}")

e2 = CompositeExpression(BinaryNonCommutativeOperator.BIT_SHIFT_LEFT, e1, t)
print(f"{e2} = {e2.evaluate()}")

True = 1
-(True) = -1
(-(True)) << (True) = -2


Unfortunately, it seems that a negative bit shift is not possible. I had assumed that these operators would be able to take in any operands, but it seems I was wrong.

There are a few ways to solve this problem:
1. Discard the bit shift operators altogether
2. Construct a new custom bit shift operator which consolidates left and right bit shifts, and then chooses the right string based on the sign
3. Deal with this as possible error case, and use try/except blocks when evaluating expressions

I don't like method one since I want to use all the available operators. Method two could be good, but it would work better with an approach which didn't depend on strings as the underlying expression representation. Maybe this would be better, but I'll play around with this idea later.

In [33]:
(True)<<((True)<<(-((~(True))<<(-(~(True))))))

115792089237316195423570985008687907853269984665640564039457584007913129639936